In [ ]:
from google.colab import drive
import os
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import RegexpTokenizer
from nltk import pos_tag

In [ ]:
drive.mount('/content/drive')
datadir = "/content/drive/My Drive/CS546Data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def read_files_to_json(folder_paths):
    data_dict = {}


    if os.path.exists('/content/minecraft_data.json'):
        with open('/content/minecraft_data.json', 'r', encoding='utf-8') as json_file:
            data_dict = json.load(json_file)

    for folder_path in folder_paths:
        for filename in os.listdir(folder_path):
            if filename.endswith(".txt"):

                topic = filename.replace(".txt", "")


                with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                    content = file.read()


                data_dict[topic] = {
                    'description': content

                }


    data_json = json.dumps(data_dict, indent=4)


    with open('/content/minecraft_data.json', 'w', encoding='utf-8') as json_file:
        json_file.write(data_json)

    return data_dict

folder_paths = [
    datadir + 'KnowledgeDatabase/GamepediaTxt/Blocks',
    datadir + 'KnowledgeDatabase/GamepediaTxt/Entity',
    datadir + 'KnowledgeDatabase/GamepediaTxt/Items',
    datadir + 'KnowledgeDatabase/GamepediaTxt/Other'
]
minecraft_data = read_files_to_json(folder_paths)

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text.lower())
    tokens = [t for t in tokens if t not in stopwords.words('english')]
    pos_tags = pos_tag(tokens)  # Get POS tags
    tokens = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in pos_tags]  # Lemmatize with POS tags
    return ' '.join(tokens)

with open('minecraft_data.json', 'r', encoding='utf-8') as f:
    minecraft_data = json.load(f)

preprocessed_data = {key: preprocess_text(value['description']) for key, value in minecraft_data.items()}

with open('preprocessed_minecraft_data.json', 'w', encoding='utf-8') as f:
    json.dump(preprocessed_data, f, indent=4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.8/412.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 7.5 MB/s eta 0:00:00


In [ ]:
es_url = "https://134.209.74.243:9200/"
from elasticsearch import Elasticsearch

es = Elasticsearch(
    ['https://134.209.74.243:9200'],
    verify_certs=False,
    http_auth=('elastic', 'mmjIugA2iaP=y0q2*5=4')
)
# es = Elasticsearch(hosts = [ES_NODES])


/usr/local/lib/python3.10/dist-packages/elasticsearch/_sync/client/__init__.py:397: SecurityWarning: Connecting to 'https://134.209.74.243:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(
<ipython-input-6-5c3c09fe588c>:4: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [ ]:
# es.indices.create(index='minecraft')

In [ ]:
for topic, description in preprocessed_data.items():
    print(f"Topic: {topic}")
    print(f"Description: {description[:50]}...")

    doc = {
        'topic': topic,
        'description': description
    }

    try:
        res = es.index(index='minecraft', id=topic, body=doc)
        print(f"Indexed {topic}")
    except Exception as e:
        print(f"Failed to index {topic}: {e}")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100

Topic: Cauldron
Description: cauldron block hold water cauldron mine use pickax...
Indexed Cauldron
Topic: Banner
Description: banner flag tall decorative block feature field hi...
Indexed Banner
Topic: Andesite
Description: andesite type igneous rock polish andesite polish ...
Indexed Andesite
Topic: Block_of_Coal
Description: block coal also know coal block mineral block also...
Indexed Block_of_Coal
Topic: Block_of_Iron
Description: block iron also know iron block decorative mineral...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100

Indexed Block_of_Iron
Topic: Cake
Description: cake food block eat player place cake cannot recol...
Indexed Cake
Topic: Chest
Description: chest block store item chest obtain craft break pr...
Indexed Chest
Topic: Cactus
Description: cactus plural cactus cactus plant occur naturally ...
Indexed Cactus
Topic: Air
Description: air block minecraft use represent unoccupied space...
Indexed Air
Topic: Carpet
Description: carpet thin block mainly use decoration carpet min...
Indexed Carpet
Topic: Block_of_Emerald
Description: block emerald also know emerald block decorative m...
Indexed Block_of_Emerald
Topic: Beacon
Description: beacon unique block project light beam skyward pro...
Indexed Beacon
Topic: Block_of_Quartz
Description: block quartz also know quartz block mineral block ...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100

...

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100

Indexed Name_Tag
Topic: Map
Description: map item use view explore terrain console edition ...
Indexed Map
Topic: Steak
Description: steak food item obtain cow cow mooshroom dy fire d...
Indexed Steak
Topic: Sugar
Description: sugar food item derive sugar cane witch may drop 0...
Indexed Sugar
Topic: Written_Book
Description: write book item create book quill sign icon write ...
Indexed Written_Book
Topic: Wheat
Description: wheat item primarily obtain farm dungeon chest cha...
Indexed Wheat
Topic: Slimeball
Description: slimeball item drop slime tiny slime drop 0 2 slim...
Indexed Slimeball
Topic: Stick
Description: stick item use craft many tool item bonus chest ma...
Indexed Stick
Topic: Sword
Description: sword melee weapon monster spawn sword 8 5 9 5 loo...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100

Indexed Sword
Topic: Spider_Eye
Description: spider eye poisonous food brewing item spider cave...
Indexed Spider_Eye
Topic: Spawn_Egg
Description: spawn egg item may use spawn mobs directly allow p...
Indexed Spawn_Egg
Topic: Shovel
Description: shovel tool use ease process collect dirt block zo...
Indexed Shovel
Topic: Shears
Description: shear tool shepherd villager sell shear 3 4 emeral...
Indexed Shears
Topic: Tools
Description: tool item use player hold perform action faster ef...
Indexed Tools


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100

In [ ]:
def search_minecraft(query):
    # Define the search request
    search_request = {
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["topic", "description"]
            }
        }
    }

    response = es.search(index="minecraft", body=search_request)

    hits = response['hits']['hits']
    results = []
    for hit in hits:
        topic = hit['_source']['topic']
        description = hit['_source']['description']
        results.append((topic, description))

    return results

query = "What is a Cauldron used for?"
results = search_minecraft(query)
for topic, description in results:
    print(f"Topic: {topic}")
    print(f"Description: {description}\n")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Topic: Cauldron
Description: cauldron block hold water cauldron mine use pickaxe mine without pickaxe drop nothing cauldron destroy water inside lose cauldron craft iron ingot single empty cauldron generate witch hut fill cauldron water press use cauldron water bucket cauldron also chance fill water rain upon water cauldron use fill glass bottle turn water bottle wash dye leather armor remove top pattern layer banner use cauldron press use cauldron glass bottle leather armor banner cauldron extinguish mob fire include player fall use include extinguish mobs cause water level cauldron decrease one third use three time empty must refill additional us endermen fill cauldron take damage water cauldron cannot use fill empty bucket water bottle cannot use refill cauldron cauldron fill water bucket nether cauldron act power source redstone comparator cauldron behind possibly separate unpowered solid block comparator output signal strength proportional full cauldron 0 empty 1 one third full 2 

In [ ]:
import nltk

question = "What is the recipe for an Enchantment Table?"

tokens = nltk.word_tokenize(question)

In [ ]:
pos_tags = nltk.pos_tag(tokens)

In [ ]:
keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

In [ ]:
index_name = 'minecraft'

def fetch_indexed_data(index_name):
    indexed_data = {}

    result = es.search(index=index_name, scroll='1m', size=1000)
    scroll_id = result['_scroll_id']
    while len(result['hits']['hits']):
        for hit in result['hits']['hits']:
            doc_id = hit['_id']
            doc_source = hit['_source']
            indexed_data[doc_id] = doc_source
        result = es.scroll(scroll_id=scroll_id, scroll='1m')
        scroll_id = result['_scroll_id']

    return indexed_data

indexed_data = fetch_indexed_data(index_name)

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.209.74.243'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [ ]:
keywords = [keyword.lower() for keyword in keywords]

# Check if any of the keywords match the keys in your indexed data
matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

documents = [data['description'] for data in relevant_data.values()]

texts = [question] + documents

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]



In [ ]:
import re
from nltk.tokenize import sent_tokenize

# Function to extract keywords from a text
def extract_keywords(text):
    return set(re.findall(r'\b\w+\b', text.lower()))

# Get the keywords from the user's question
question_keywords = extract_keywords(question)

# Initialize an empty list to store sentences and their scores
candidate_answers = []

# Go through each document in the ranked list
for doc in document_ranking:
    # Tokenize the document into sentences
    sentences = sent_tokenize(doc)

    # Go through each sentence in the document
    for sentence in sentences:
        # Get the keywords in the sentence
        sentence_keywords = extract_keywords(sentence)

        # Compute a score based on keyword overlap
        score = len(question_keywords.intersection(sentence_keywords))

        # Append the sentence and its score to the list of candidates
        candidate_answers.append((sentence, score))

# Sort the candidate answers by their scores in descending order
candidate_answers.sort(key=lambda x: x[1], reverse=True)

# The top-ranked sentence is your answer (or you could return the top N sentences)
answer = candidate_answers[0][0]

# Print the answer
print(answer)

enchantment table block allow player spend experience point level enchant tool book armor enchantment table mine use pickaxe mine without pickaxe drop nothing enchantment table 4 block high player walk edge sneak cannot move piston enchantment table place ice player slide though ice block like slab enchant table main purpose enchant item bookshelf surround table block air increase maximum enchantment level table enchant tool armor except hoe shear flint steel lead horse armor hoe shear cannot enchant enchantment table require anvil appropriate enchant book arcane glyph float bookshelves enchant table cryptic rune enchantment table interface write standard galactic alphabet simple alphabet substitution cipher use commander keen series computer game run randomly construct follow list word elder scroll klaatu berata niktu xyzzy bless curse light darkness fire air earth water hot dry cold wet ignite snuff embiggen twist shorten stretch fiddle destroy imbue galvanize enchant free limited ra

In [ ]:
def get_answer_baseline(question):
  tokens = nltk.word_tokenize(question)

  pos_tags = nltk.pos_tag(tokens)

  keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

  keywords = [keyword.lower() for keyword in keywords]

  # Check if any of the keywords match the keys in your indexed data
  matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
  relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

  documents = [data['description'] for data in relevant_data.values()]

  texts = [question] + documents

  vectorizer = TfidfVectorizer()
  tfidf_matrix = vectorizer.fit_transform(texts)

  cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

  document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]

  question_keywords = extract_keywords(question)

  # Initialize an empty list to store sentences and their scores
  candidate_answers = []

  # Go through each document in the ranked list
  for doc in document_ranking:
      # Tokenize the document into sentences
      sentences = sent_tokenize(doc)

      # Go through each sentence in the document
      for sentence in sentences:
          # Get the keywords in the sentence
          sentence_keywords = extract_keywords(sentence)

          # Compute a score based on keyword overlap
          score = len(question_keywords.intersection(sentence_keywords))

          # Append the sentence and its score to the list of candidates
          candidate_answers.append((sentence, score))

  # Sort the candidate answers by their scores in descending order
  candidate_answers.sort(key=lambda x: x[1], reverse=True)

  # The top-ranked sentence is your answer (or you could return the top N sentences)
  try:
    answer = candidate_answers[0][0]
  except Exception as e:
    print(f"An error occurred: {e}")
    return "None"
  # Print the answer
  print(answer)
  return answer

In [ ]:
get_answer_baseline("What do you need to make a Nether Portal?")

netherrack formerly know netherstone rock like block find nether netherrack collect pickaxe netherrack make nether terrain pocket edition netherrack create nether reactor activate netherrack top lit fire burn infinitely netherrack destroyed hand 2 second make stone destroy least amount time netherrack mine 50 millisecond golden pickaxe diamond pickaxe enchant level efficiency however 25 second delay mining block instantly destroy efficiency ii diamond pickaxe well 25 second delay time need mine le one game tick game code reference hellrock tutorial mob farm tutorial trap


'netherrack formerly know netherstone rock like block find nether netherrack collect pickaxe netherrack make nether terrain pocket edition netherrack create nether reactor activate netherrack top lit fire burn infinitely netherrack destroyed hand 2 second make stone destroy least amount time netherrack mine 50 millisecond golden pickaxe diamond pickaxe enchant level efficiency however 25 second delay mining block instantly destroy efficiency ii diamond pickaxe well 25 second delay time need mine le one game tick game code reference hellrock tutorial mob farm tutorial trap'

In [ ]:
get_answer_baseline("What is the recipe for an Enchantment Table?")

enchantment table block allow player spend experience point level enchant tool book armor enchantment table mine use pickaxe mine without pickaxe drop nothing enchantment table 4 block high player walk edge sneak cannot move piston enchantment table place ice player slide though ice block like slab enchant table main purpose enchant item bookshelf surround table block air increase maximum enchantment level table enchant tool armor except hoe shear flint steel lead horse armor hoe shear cannot enchant enchantment table require anvil appropriate enchant book arcane glyph float bookshelves enchant table cryptic rune enchantment table interface write standard galactic alphabet simple alphabet substitution cipher use commander keen series computer game run randomly construct follow list word elder scroll klaatu berata niktu xyzzy bless curse light darkness fire air earth water hot dry cold wet ignite snuff embiggen twist shorten stretch fiddle destroy imbue galvanize enchant free limited ra

'enchantment table block allow player spend experience point level enchant tool book armor enchantment table mine use pickaxe mine without pickaxe drop nothing enchantment table 4 block high player walk edge sneak cannot move piston enchantment table place ice player slide though ice block like slab enchant table main purpose enchant item bookshelf surround table block air increase maximum enchantment level table enchant tool armor except hoe shear flint steel lead horse armor hoe shear cannot enchant enchantment table require anvil appropriate enchant book arcane glyph float bookshelves enchant table cryptic rune enchantment table interface write standard galactic alphabet simple alphabet substitution cipher use commander keen series computer game run randomly construct follow list word elder scroll klaatu berata niktu xyzzy bless curse light darkness fire air earth water hot dry cold wet ignite snuff embiggen twist shorten stretch fiddle destroy imbue galvanize enchant free limited r

In [ ]:
get_answer_baseline("What is the use of an Enchantment Table?")

enchantment table block allow player spend experience point level enchant tool book armor enchantment table mine use pickaxe mine without pickaxe drop nothing enchantment table 4 block high player walk edge sneak cannot move piston enchantment table place ice player slide though ice block like slab enchant table main purpose enchant item bookshelf surround table block air increase maximum enchantment level table enchant tool armor except hoe shear flint steel lead horse armor hoe shear cannot enchant enchantment table require anvil appropriate enchant book arcane glyph float bookshelves enchant table cryptic rune enchantment table interface write standard galactic alphabet simple alphabet substitution cipher use commander keen series computer game run randomly construct follow list word elder scroll klaatu berata niktu xyzzy bless curse light darkness fire air earth water hot dry cold wet ignite snuff embiggen twist shorten stretch fiddle destroy imbue galvanize enchant free limited ra

'enchantment table block allow player spend experience point level enchant tool book armor enchantment table mine use pickaxe mine without pickaxe drop nothing enchantment table 4 block high player walk edge sneak cannot move piston enchantment table place ice player slide though ice block like slab enchant table main purpose enchant item bookshelf surround table block air increase maximum enchantment level table enchant tool armor except hoe shear flint steel lead horse armor hoe shear cannot enchant enchantment table require anvil appropriate enchant book arcane glyph float bookshelves enchant table cryptic rune enchantment table interface write standard galactic alphabet simple alphabet substitution cipher use commander keen series computer game run randomly construct follow list word elder scroll klaatu berata niktu xyzzy bless curse light darkness fire air earth water hot dry cold wet ignite snuff embiggen twist shorten stretch fiddle destroy imbue galvanize enchant free limited r

In [ ]:
get_answer_baseline("How do I use an Enchantment Table?")

enchantment table block allow player spend experience point level enchant tool book armor enchantment table mine use pickaxe mine without pickaxe drop nothing enchantment table 4 block high player walk edge sneak cannot move piston enchantment table place ice player slide though ice block like slab enchant table main purpose enchant item bookshelf surround table block air increase maximum enchantment level table enchant tool armor except hoe shear flint steel lead horse armor hoe shear cannot enchant enchantment table require anvil appropriate enchant book arcane glyph float bookshelves enchant table cryptic rune enchantment table interface write standard galactic alphabet simple alphabet substitution cipher use commander keen series computer game run randomly construct follow list word elder scroll klaatu berata niktu xyzzy bless curse light darkness fire air earth water hot dry cold wet ignite snuff embiggen twist shorten stretch fiddle destroy imbue galvanize enchant free limited ra

'enchantment table block allow player spend experience point level enchant tool book armor enchantment table mine use pickaxe mine without pickaxe drop nothing enchantment table 4 block high player walk edge sneak cannot move piston enchantment table place ice player slide though ice block like slab enchant table main purpose enchant item bookshelf surround table block air increase maximum enchantment level table enchant tool armor except hoe shear flint steel lead horse armor hoe shear cannot enchant enchantment table require anvil appropriate enchant book arcane glyph float bookshelves enchant table cryptic rune enchantment table interface write standard galactic alphabet simple alphabet substitution cipher use commander keen series computer game run randomly construct follow list word elder scroll klaatu berata niktu xyzzy bless curse light darkness fire air earth water hot dry cold wet ignite snuff embiggen twist shorten stretch fiddle destroy imbue galvanize enchant free limited r

In [ ]:
get_answer_baseline("How to make a Potion of Healing?")

potion brewable item imbue consumer specific time limited effect minecraft witch drop potion heal fire resistance swiftness water breathing die drink potion water bottle may catch junk fish use glass bottle water source block cauldron turn water bottle use glass bottle cauldron also low water level cauldron 1 3 almost potion obtain brew start water bottle base potion different effect accord damage value exception revert potion identical base potion potion use press hold use similarly eat food upon use apply correspond status effect player potion four tier base extend level ii revert potion extend obtain add piece redstone potion brewing stand level ii require glowstone dust extend potion receive glowstone dust level ii potion receive piece redstone change effect available become reverted potion potion also corrupt use ferment spider eye usually invert effect example heal harm corrupt potion four tier normal potion tier usually keep corruption console edition bar underneath potion one m

'potion brewable item imbue consumer specific time limited effect minecraft witch drop potion heal fire resistance swiftness water breathing die drink potion water bottle may catch junk fish use glass bottle water source block cauldron turn water bottle use glass bottle cauldron also low water level cauldron 1 3 almost potion obtain brew start water bottle base potion different effect accord damage value exception revert potion identical base potion potion use press hold use similarly eat food upon use apply correspond status effect player potion four tier base extend level ii revert potion extend obtain add piece redstone potion brewing stand level ii require glowstone dust extend potion receive glowstone dust level ii potion receive piece redstone change effect available become reverted potion potion also corrupt use ferment spider eye usually invert effect example heal harm corrupt potion four tier normal potion tier usually keep corruption console edition bar underneath potion one 

In [ ]:
get_answer_baseline("What should I add to a Base Potion to obtain a Potion of Healing?")

potion brewable item imbue consumer specific time limited effect minecraft witch drop potion heal fire resistance swiftness water breathing die drink potion water bottle may catch junk fish use glass bottle water source block cauldron turn water bottle use glass bottle cauldron also low water level cauldron 1 3 almost potion obtain brew start water bottle base potion different effect accord damage value exception revert potion identical base potion potion use press hold use similarly eat food upon use apply correspond status effect player potion four tier base extend level ii revert potion extend obtain add piece redstone potion brewing stand level ii require glowstone dust extend potion receive glowstone dust level ii potion receive piece redstone change effect available become reverted potion potion also corrupt use ferment spider eye usually invert effect example heal harm corrupt potion four tier normal potion tier usually keep corruption console edition bar underneath potion one m

'potion brewable item imbue consumer specific time limited effect minecraft witch drop potion heal fire resistance swiftness water breathing die drink potion water bottle may catch junk fish use glass bottle water source block cauldron turn water bottle use glass bottle cauldron also low water level cauldron 1 3 almost potion obtain brew start water bottle base potion different effect accord damage value exception revert potion identical base potion potion use press hold use similarly eat food upon use apply correspond status effect player potion four tier base extend level ii revert potion extend obtain add piece redstone potion brewing stand level ii require glowstone dust extend potion receive glowstone dust level ii potion receive piece redstone change effect available become reverted potion potion also corrupt use ferment spider eye usually invert effect example heal harm corrupt potion four tier normal potion tier usually keep corruption console edition bar underneath potion one 

In [ ]:
get_answer_baseline("Which reagent is used for the Potion of Healing?")

potion brewable item imbue consumer specific time limited effect minecraft witch drop potion heal fire resistance swiftness water breathing die drink potion water bottle may catch junk fish use glass bottle water source block cauldron turn water bottle use glass bottle cauldron also low water level cauldron 1 3 almost potion obtain brew start water bottle base potion different effect accord damage value exception revert potion identical base potion potion use press hold use similarly eat food upon use apply correspond status effect player potion four tier base extend level ii revert potion extend obtain add piece redstone potion brewing stand level ii require glowstone dust extend potion receive glowstone dust level ii potion receive piece redstone change effect available become reverted potion potion also corrupt use ferment spider eye usually invert effect example heal harm corrupt potion four tier normal potion tier usually keep corruption console edition bar underneath potion one m

'potion brewable item imbue consumer specific time limited effect minecraft witch drop potion heal fire resistance swiftness water breathing die drink potion water bottle may catch junk fish use glass bottle water source block cauldron turn water bottle use glass bottle cauldron also low water level cauldron 1 3 almost potion obtain brew start water bottle base potion different effect accord damage value exception revert potion identical base potion potion use press hold use similarly eat food upon use apply correspond status effect player potion four tier base extend level ii revert potion extend obtain add piece redstone potion brewing stand level ii require glowstone dust extend potion receive glowstone dust level ii potion receive piece redstone change effect available become reverted potion potion also corrupt use ferment spider eye usually invert effect example heal harm corrupt potion four tier normal potion tier usually keep corruption console edition bar underneath potion one 

In [ ]:
get_answer_baseline("Is Obsidian blast resistant?")

prismarine stone like material appear underwater ocean monument prismarine brick decorative form prismarine sawtooth edge brick pattern dark prismarine decorative form prismarine dark tight grid pattern type prismarine mine use pickaxe drop mine without pickaxe block lose type prismarine naturally generate main block make ocean monument normal prismarine brick find large amount due size structure prismarine brick make 23 pillar monument appear decoratively throughout structure somewhat rare dark prismarine encase 8 gold block reward dungeon appear few decorative feature type prismarine craft use vary amount prismarine shard ink sac case dark prismarine resource obtain underwater mobs current usage prismarine decoration high blast resistance building block normal prismarine uniquely useful decoration animate texture crack prismarine slowly cycle color aqua green purple indigo full animation prismarine texture change color 22 time irregular sequence whole pattern repeat take 5 minute 30 

'prismarine stone like material appear underwater ocean monument prismarine brick decorative form prismarine sawtooth edge brick pattern dark prismarine decorative form prismarine dark tight grid pattern type prismarine mine use pickaxe drop mine without pickaxe block lose type prismarine naturally generate main block make ocean monument normal prismarine brick find large amount due size structure prismarine brick make 23 pillar monument appear decoratively throughout structure somewhat rare dark prismarine encase 8 gold block reward dungeon appear few decorative feature type prismarine craft use vary amount prismarine shard ink sac case dark prismarine resource obtain underwater mobs current usage prismarine decoration high blast resistance building block normal prismarine uniquely useful decoration animate texture crack prismarine slowly cycle color aqua green purple indigo full animation prismarine texture change color 22 time irregular sequence whole pattern repeat take 5 minute 30

In [ ]:
get_answer_baseline("How to mine Stone?")

stone block find abundance overworld stone require pickaxe mined case drop cobblestone mine without pickaxe drop nothing stone mine silk touch enchant pickaxe drop stone make majority solid block generate chunk generate stone find layer grass dirt biome also see expose side mountains extreme hill biomes world generate new chunk stone become ore almost every cavern mineshaft generate stone level stone also make lava flow top water water replace silverfish ability enter hide stone create monster egg stone id name minecraft stone share andesite diorite granite differentiate block data stone also block state expect replace functionality block data future version stone presumably igneous rock due fact form rapid cool magma side circle cave make stone various type stone since bountiful update


'stone block find abundance overworld stone require pickaxe mined case drop cobblestone mine without pickaxe drop nothing stone mine silk touch enchant pickaxe drop stone make majority solid block generate chunk generate stone find layer grass dirt biome also see expose side mountains extreme hill biomes world generate new chunk stone become ore almost every cavern mineshaft generate stone level stone also make lava flow top water water replace silverfish ability enter hide stone create monster egg stone id name minecraft stone share andesite diorite granite differentiate block data stone also block state expect replace functionality block data future version stone presumably igneous rock due fact form rapid cool magma side circle cave make stone various type stone since bountiful update'

In [ ]:
!pip install pandas

In [ ]:
import json
import pandas as pd

with open('./final_minecraft_dataset.json', 'r') as file:
    data = json.load(file)

qa_data = []
for entry in data['data']:
    title = entry['title']
    for paragraph in entry['paragraphs']:
        context = paragraph['context']
        for qa in paragraph['qas']:
            question = qa['question']
            id = qa['id']
            for answer in qa['answers']:
                answer_text = answer['text']
                answer_start = answer['answer_start']
                qa_data.append({'title': title, 'context': context, 'question': question, 'id': id, 'answer_text': answer_text, 'answer_start': answer_start})

df = pd.DataFrame(qa_data)

In [ ]:
df

,title,context,question,id,answer_text,answer_start
0,Cauldron,entity cauldron entity entity block entity hol...,How can an entity cauldron entity be destroyed...,q1,mine use pickaxe mine without pickaxe drop not...,98
1,Cauldron,entity cauldron entity entity block entity hol...,What happens when you use an entity cauldron e...,q2,turn water bottle wash dye entity leather enti...,433
2,Cauldron,entity cauldron entity entity block entity hol...,How does the entity cauldron entity act as a p...,q3,act power source redstone comparator entity ca...,1132
3,Banner,banner flag tall decorative block feature fiel...,How many basic colors can a blank flag have in...,q1,16 color,235
4,Banner,banner flag tall decorative block feature fiel...,How many patterns can be overlaid on a Minecra...,q2,six pattern,675
...,...,...,...,...,...,...
965,Shears,shear entity tool entity shepherd villager sel...,"Besides wool, what can shears also be used to ...",q2,harvest leaf tall grass fern normal 2 high dea...,268
966,Shears,shear entity tool entity shepherd villager sel...,What happens when a mooshroom is sheared?,q3,drop 5 red mushroom turn normal cow,658
967,Tools,tool item use entity player entity held perfor...,What are some examples of tools that do not st...,q1,"tool include hoe, bow, fishing rod, carrot on ...",271
968,Tools,tool item use entity player entity held perfor...,"What is an exception to the tool use, for enti...",q2,"exception entity clock, entity compass",159


In [ ]:
pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.6 MB/s eta 0:00:00


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.3 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=22f1d9b49abee4048a12c1ff940443aa62fac6b37121ac59c8badbc9c02d30a7
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

import numpy as np


model = SentenceTransformer('bert-base-nli-mean-tokens')

.gitattributes:   0%|          | 0.00/391 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/399 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

In [ ]:
questions = df['question'].head(50).to_list()
contexts = df['context'].head(50).to_list()
references = df['answer_text'].head(50).to_list()
candidates = []
for i in range (50):
    question = questions[i]
    context = contexts[i]
    predicted_answer = get_answer_baseline(question)
    gt = references[i]
    print("Q:", question)
    print("A:", predicted_answer)
    print("GT:", gt)
    candidates.append(predicted_answer)

predicted_answer_embeddings = model.encode(candidates)

reference_answer_embeddings = model.encode(references)

similarities = [cosine_similarity([pred], [ref])[0][0] for pred, ref in zip(predicted_answer_embeddings, reference_answer_embeddings)]

average_similarity = np.mean(similarities)

print("Average Semantic Similarity:", average_similarity)


# P, R, F1 = score(candidates, references, lang="en", verbose=True)
# print(P.mean())
# print(R.mean())
# print(F1.mean())

water natural fluid water cannot obtain item matter retrieve bucket water naturally generate overworld ocean lakes river spring village desert well player water take 5 time normal amount time mine block ground 25 time ungrounded water spread across surface slightly realistic way cannot spread top another water block water source block create spring generates flow fall water spring destroy flowing water create slowly dry hole six block few point origin water tend flow direction source block fall water land solid block always flow least one direction unless restricted water travel distance 8 block counter reset 0 every time land low level block next water source block destroy water immediately flow direction unless block beneath destroy block block next water update calculates flow spread water extinguish fire wash plant snow torch carpet tripwire rail cobweb flow drop item water source block appear horizontally orthogonally next 2 source block top solid block water source block principl

In [ ]:
import numpy as np


threshold = 0.5


matches = [similarity >= threshold for similarity in similarities]

# Calculate Precision, Recall, F1
true_positives = sum(matches)
predicted_positives = len(candidates)  # Total number of predicted answers
actual_positives = len(references)  # Total number of reference answers

precision = true_positives / predicted_positives if predicted_positives else 0
recall = true_positives / actual_positives if actual_positives else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

Precision: 0.34
Recall: 0.34
F1: 0.34


In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the pretrained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

def get_answer(question):
    tokens = nltk.word_tokenize(question)

    pos_tags = nltk.pos_tag(tokens)

    keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

    keywords = [keyword.lower() for keyword in keywords]

    # Check if any of the keywords match the keys in your indexed data
    matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
    relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

    documents = [data['description'] for data in relevant_data.values()]

    texts = [question] + documents

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(texts)

    cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

    document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
    try:
        most_relevant_doc = document_ranking[0]
    except Exception as e:
        print(f"An error occurred: {e}")
        return "None"
    context = most_relevant_doc
    # Tokenize the input with truncation
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt", truncation=True, max_length=512)

    # Debugging: Print the length of the input
    print("Length of input:", inputs["input_ids"].size(1))

    # Run the model
    with torch.no_grad():
        outputs = model(inputs["input_ids"], token_type_ids=inputs["token_type_ids"])

    # Get the most likely start and end positions for the answer
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)

    # Debugging: Print the start and end positions
    print("Start position:", answer_start)
    print("End position:", answer_end)

    # Check if the answer span makes sense
    if answer_end >= answer_start:
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end+1]))
    else:
        answer = "Answer not found in the text."

    return answer

# Assume the first document in the ranking is the most relevant
most_relevant_doc = document_ranking[0]

# Get the answer from the most relevant document
answer = get_answer(question)
print(answer)

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Length of input: 130
Start position: tensor(48)
End position: tensor(49)
open press


In [ ]:
question = "What is the use of a redstone comparator in Minecraft?"
# tokens = nltk.word_tokenize(question)

# pos_tags = nltk.pos_tag(tokens)

# keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

# keywords = [keyword.lower() for keyword in keywords]

# # Check if any of the keywords match the keys in your indexed data
# matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
# relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

# documents = [data['description'] for data in relevant_data.values()]

# texts = [question] + documents

# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(texts)

# cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

# document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
# most_relevant_doc = document_ranking[0]

answer = get_answer(question)
print(answer)

Length of input: 227
Start position: tensor(40)
End position: tensor(49)
pickaxe mine without pickaxe drop nothing


In [ ]:
# question = "How can I create a Splash Potion of Healing?"
# tokens = nltk.word_tokenize(question)

# pos_tags = nltk.pos_tag(tokens)

# keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

# keywords = [keyword.lower() for keyword in keywords]

# # Check if any of the keywords match the keys in your indexed data
# matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
# relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

# documents = [data['description'] for data in relevant_data.values()]

# texts = [question] + documents

# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(texts)

# cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

# document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
# most_relevant_doc = document_ranking[0]

# print("Most relevant document:", most_relevant_doc)
answer = get_answer(question)
print("Answer:", answer)

Length of input: 333
Start position: tensor(45)
End position: tensor(41)
Answer: Answer not found in the text.


In [ ]:
question = "How do I get Obsidian?"
tokens = nltk.word_tokenize(question)

pos_tags = nltk.pos_tag(tokens)

keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

keywords = [keyword.lower() for keyword in keywords]

# Check if any of the keywords match the keys in your indexed data
matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

documents = [data['description'] for data in relevant_data.values()]

texts = [question] + documents

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
most_relevant_doc = document_ranking[0]

print("Most relevant document:", most_relevant_doc)
answer = get_answer(question, most_relevant_doc)
print("Answer:", answer)

Most relevant document: gold nugget item obtain kill zombie pigman zombie pigman kill may drop 0 1 gold nugget kill use weapon enchant looting may drop additional gold nugget per level looting use maximum 4 gold nugget use craft golden carrot glistering melon brewing also gold nugget combine gunpowder dye create firework star burst star shape explosion


TypeError: ignored

In [ ]:
question = "What do you need to mine a Diamond?"
tokens = nltk.word_tokenize(question)

pos_tags = nltk.pos_tag(tokens)

keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

keywords = [keyword.lower() for keyword in keywords]

# Check if any of the keywords match the keys in your indexed data
matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

documents = [data['description'] for data in relevant_data.values()]

texts = [question] + documents

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
most_relevant_doc = document_ranking[0]

print("Most relevant document:", most_relevant_doc)
answer = get_answer(question, most_relevant_doc)
print("Answer:", answer)

In [ ]:
question = "What is the most rare Ore in minecraft?"
tokens = nltk.word_tokenize(question)

pos_tags = nltk.pos_tag(tokens)

keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

keywords = [keyword.lower() for keyword in keywords]

# Check if any of the keywords match the keys in your indexed data
matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

documents = [data['description'] for data in relevant_data.values()]

texts = [question] + documents

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
most_relevant_doc = document_ranking[0]

print("Most relevant document:", most_relevant_doc)
answer = get_answer(question, most_relevant_doc)
print("Answer:", answer)

In [ ]:
question = "Is Obsidian blast resistant?"
tokens = nltk.word_tokenize(question)

pos_tags = nltk.pos_tag(tokens)

keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

keywords = [keyword.lower() for keyword in keywords]

# Check if any of the keywords match the keys in your indexed data
matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

documents = [data['description'] for data in relevant_data.values()]

texts = [question] + documents

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
most_relevant_doc = document_ranking[0]

print("Most relevant document:", most_relevant_doc)
answer = get_answer(question, most_relevant_doc)
print("Answer:", answer)

In [ ]:
question = "How to mine Stone?"
tokens = nltk.word_tokenize(question)

pos_tags = nltk.pos_tag(tokens)

keywords = [word for word, pos in pos_tags if pos in ('NN', 'NNS', 'NNP', 'NNPS', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ')]

keywords = [keyword.lower() for keyword in keywords]

# Check if any of the keywords match the keys in your indexed data
matching_entities = [entity for entity in indexed_data if any(keyword in entity.lower() for keyword in keywords)]
relevant_data = {entity: indexed_data[entity] for entity in matching_entities}

documents = [data['description'] for data in relevant_data.values()]

texts = [question] + documents

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(texts)

cosine_similarities = linear_kernel(tfidf_matrix[0:1], tfidf_matrix).flatten()

document_ranking = [documents[i - 1] for i in cosine_similarities.argsort()[::-1] if i != 0]
most_relevant_doc = document_ranking[0]

print("Most relevant document:", most_relevant_doc)
answer = get_answer(question, most_relevant_doc)
print("Answer:", answer)

In [ ]:
from sentence_transformers import SentenceTransformer

from sklearn.metrics.pairwise import cosine_similarity

import numpy as np




In [ ]:
questions = df['question'].head(50).to_list()
contexts = df['context'].head(50).to_list()
references = df['answer_text'].head(50).to_list()
candidates = []
for i in range (50):
    question = questions[i]
    context = contexts[i]
    predicted_answer = get_answer(question)
    gt = references[i]
    print("Q:", question)
    print("A:", predicted_answer)
    print("GT:", gt)
    candidates.append(predicted_answer)


model = SentenceTransformer('bert-base-nli-mean-tokens')
predicted_answer_embeddings = model.encode(candidates)

reference_answer_embeddings = model.encode(references)

similarities = [cosine_similarity([pred], [ref])[0][0] for pred, ref in zip(predicted_answer_embeddings, reference_answer_embeddings)]

average_similarity = np.mean(similarities)

print("Average Semantic Similarity:", average_similarity)


# P, R, F1 = score(candidates, references, lang="en", verbose=True)
# print(P.mean())
# print(R.mean())
# print(F1.mean())

Length of input: 333


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Start position: tensor(45)
End position: tensor(41)
Q: How can an entity cauldron entity be destroyed, and what happens to the water inside?
A: Answer not found in the text.
GT: mine use pickaxe mine without pickaxe drop nothing entity cauldron entity destroyed water inside lose
Length of input: 512


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Start position: tensor(0)
End position: tensor(0)
Q: What happens when you use an entity cauldron entity filled with water on an entity leather entity entity armor entity or a banner?
A: [CLS]
GT: turn water bottle wash dye entity leather entity entity armor entity remove top pattern layer banner use entity cauldron entity press use entity cauldron entity
Length of input: 512


...

Start position: tensor(48)
End position: tensor(48)
Q: What can be used to change the texture and shape of blocks in Minecraft?
A: model
GT: use resource pack entity player entity change texture block include whether texture animate also change shape block use model
Length of input: 512


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Start position: tensor(0)
End position: tensor(0)
Q: What is the most efficient tool for mining a entity clay entity block?
A: [CLS]
GT: shovel
Length of input: 512


Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


Start position: tensor(0)
End position: tensor(0)
Q: How many entity clay entity balls can one obtain from breaking a entity clay entity block?
A: [CLS]
GT: 4 entity clay entity balls
Length of input: 512
Start position: tensor(0)
End position: tensor(0)
Q: In what kind of patterns are entity clay entity blocks found naturally?
A: [CLS]
GT: circular pattern
Length of input: 131
Start position: tensor(45)
End position: tensor(48)
Q: What is the main use of a entity brewing stand entity?
A: allow brew potion interface
GT: allow brewing of potion
Length of input: 130
Start position: tensor(48)
End position: tensor(49)
Q: How can a entity brewing stand entity be mined effectively?
A: open press
GT: use pickaxe
Average Semantic Similarity: 0.5039503


In [ ]:
import numpy as np

threshold = 0.5


matches = [similarity >= threshold for similarity in similarities]

# Calculate Precision, Recall, F1
true_positives = sum(matches)
predicted_positives = len(candidates)  # Total number of predicted answers
actual_positives = len(references)  # Total number of reference answers

precision = true_positives / predicted_positives if predicted_positives else 0
recall = true_positives / actual_positives if actual_positives else 0
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) else 0

print("Precision:", precision)
print("Recall:", recall)
print("F1:", f1)

Precision: 0.44
Recall: 0.44
F1: 0.44
